<a href="https://colab.research.google.com/github/Kroojel/INT-Group-16/blob/main/INT2_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we need to import the classes

In [2]:
# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# Data transformation imports
from torchvision import transforms

# General Libraries 
import matplotlib.pyplot as plt
import numpy as np

# Loss fucntion and optimiser imports
import torch.optim

Preparing cuda (if available)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

Loading the dataset and the ten labels "classes" relating to the ten classifications

In [5]:
batchSize = 128

# Taining set:

trainTransform = transforms.Compose(
    [
     transforms.ToTensor(),                                 # Turns image into tensor
     transforms.RandomAdjustSharpness(2, 0.5),              # 50% chance of increasing sharpness
     transforms.RandomAdjustSharpness(0, 0.5),              # 50% chance of increasing blur
     transforms.RandomAutocontrast(0.5),                    # 50% chance of increasing contrast
     transforms.RandomHorizontalFlip(0.5),                  # 50% chance of flipping the image horizontally
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalising the tensor
    ]
)

trainSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=trainTransform)

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True, num_workers=2)

# Testing set:

testTransform = transforms.Compose(
    [
     transforms.ToTensor(),                                 # Turns image into tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalising the tensor
    ]
)

testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=testTransform)

testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

RuntimeError: ignored

Defining neural network shape:

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.prep1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1),
        )


        self.blockPrep1 = nn.Sequential(
            nn.BatchNorm2d(64),
            nn.ReLU(True),
        )

        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
        )


        self.blockPrep2 = nn.Sequential(
            nn.BatchNorm2d(64),
            nn.ReLU(True),
        )

        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
        )

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)


        self.blockPrep3 = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.ReLU(True),
        )

        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
        )


        self.blockPrep4 = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.ReLU(True),
        )

        self.branch4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
        )
        
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)


        self.blockPrep5 = nn.Sequential(
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.branch5 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
        )
        
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1)


        self.outLayer = nn.Sequential(
            nn.Linear(25088, 512),

        )

    def forward(self, input):
        input= input.to(device) # May remove after testing

          # 3 -> 64
        # Preperation
        output = self.prep1(input)

          # 64 -> 64

        # Layer 1
        output = self.blockPrep1(output)
        output = self.branch1(output) + output

        output = self.blockPrep1(output)
        output = self.branch1(output) + output
        

          # 64 -> 128

        # Layer 2
        output = self.blockPrep2(output)
        output = self.branch2(output) + self.conv2(output)
        
          # 128 -> 128

        output = self.blockPrep3(output)
        output = self.branch3(output) + output
        
          # 128 -> 256

        # Layer 3
        output = self.blockPrep4(output)
        output = self.branch4(output) + self.conv4(output)
        
          # 256 -> 256

        output = self.blockPrep5(output)
        output = self.branch5(output) + output

        # Layer 4
        output = self.blockPrep5(output)
        output = self.branch5(output) + self.conv5(output)

        output = self.blockPrep5(output)
        output = self.branch5(output) + output

        # Output
        outputSplit1 = nn.AvgPool2d(kernel_size=4, stride=4, padding=0, ceil_mode=False, count_include_pad=True)(output)
        outputSplit2 = nn.MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, return_indices=False, ceil_mode=False)(output)
        output = torch.cat( ( outputSplit1, outputSplit2 ), 1 )

        output = output.view(-1, 25088) # flattens output

        output = self.outLayer(output)

        return output 


net = Net()

Define optimiser and loss function

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.0001)

Network training and testing:

In [ ]:
for epoch in range(40):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 20 == 19:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs4
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

print('Finished Training')